In [45]:
from scipy.spatial import distance as dist
import numpy as np
import cv2
import dlib
from imutils import face_utils

In [46]:
"""eyes coordinates will be like ---

  1  2 
0      3
  5  4
  
  eye_aspect_ratio = ||1-5||+||2-4||/2*||0-3||"""

def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])
	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)
	# return the eye aspect ratio
	return ear

In [26]:
def rect_2_bb(rect):
    """dlib produce rectangle but we are familier with
    (x,y,w,h) format to draw bounding box"""
    x=rect.left()
    y=rect.top()
    w=rect.right()-x
    h=rect.bottom()-y
    return (x,y,w,h)

def shape_2_np(shape):
    """as face landmark detector can be of 68 points and 198 points
    in this notebook we are using 68 points detector"""
    coords=np.zeros((68,2),dtype="int")
    
    for i in range(68):
        coords[i]=(shape.part(i).x,shape.part(i).y)
    return coords

In [49]:
"""
THRESHOLD-----< RATIO will be considerd as blink
"""


threshold=0.27
conc_frame=3
count=0
total=0

detector=dlib.get_frontal_face_detector()
pred=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) =face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
cap=cv2.VideoCapture(0)

while True:
    ret,frame=cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    rects=detector(gray,0)
    for r in rects:
        shape=pred(gray,r)
        
        shape=shape_2_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye= shape[rStart:rEnd]
        leftEAR =  eye_aspect_ratio(leftEye)
        rightEAR =  eye_aspect_ratio(rightEye)
        
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        if ear<threshold:
            count+=1
        else:
            if count>=conc_frame:
                total+=1
                
                count=0
        cv2.putText(frame, "Blinks: {}".format(total), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
    cv2.imshow("meme",frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [43]:
cap.release()
cv2.destroyAllWindows()